# Week 7 Assignment 
For this assignment, we have one zip file question, three regex patterns, and one longer question doing some sentiment analysis. 

Question 5 is a three parter that requires you to use the output of the 1st part to complete the 2nd and the output of the 2nd part to complete the third. At the bottom of this notebook, I've put down the answers to parts 1 and 2 - so if you get stuck on one part you still have the input you need for the next one. We covered the sentiment analysis stuff pretty quickly in class, but everything you need for reference should be covered in the in_class_exercises file. 


Total Points: 100

In [1]:
import os
from zipfile import ZipFile
import io
import requests
from pprint import pprint
import re

### 1. (20 pts) Handling zip files

Open file1.zip, read in the compressed csv, and print out the lines of that file. 


In [2]:
with ZipFile('transaction_zips/' + 'file1.zip', 'r') as zf:  
        zipped_files = zf.namelist()
        
        for file_name in zipped_files:
            print(file_name)
            input_file = zf.open(file_name, 'r') 
            input_file = io.TextIOWrapper(input_file,encoding="utf-8")
            for line in input_file:
                print(line)

file1.csv
﻿CustomerId,SalesOrderId,TransactionDate,GrandTotal,Status

1,36777536,6/1/21,180,fulfilled

2,36914274,6/1/21,119.94,fulfilled

3,37054577,6/1/21,119.95,returned

4,37137392,6/1/21,132.44,fulfilled

5,37075736,6/1/21,158.9,fulfilled


#### Regex Cheatsheet

In [3]:
### Here are some patterns to reference - these are also in the slides

# ^ - matches if a string starts with a character
# $ - matches if a string ends with a character
# . - matches any character
# [] - matches if any characters are present
# + - matches one or more occurrences to the left
# ? - matches zero or one occurrence to the left
# | - used for alteration

# \A - matches if characters are at the beginning or end of the string
# \b - matches if characters are at the beginning or end of a word
# \d - matches any number
# \D - matches any non-numeric characters
# \s - matches any string that contains whitespace
# \w - matches any alphanumeric character

# \ - escapes any symbol you want to match
# [^ ] - the caret in brackets will reverse whatever pattern follows it 

### 2. (15 pts) Regex

Enter a pattern in compile() that replaces abc in the string below.
In other words, you should search for a 3 character pattern that begins and ends with a non-numeric value.

In [4]:
# multiple ways to solve all of these

test_string = 'abc ab1 bc2 cd3'
pattern = re.compile(r'\D')
pattern.sub('',test_string)

'123'

### 3. (15 pts) Regex

Enter a pattern in compile() that replaces the new line character (\n) with a space.


In [5]:
test_string = 'This\nis\nan\nissue'
pattern = re.compile(r'\n')
pattern.sub(' ',test_string)

'This is an issue'

### 4. (15 pts) Regex

Enter a pattern in compile() that replaces the missspelled emails.

In [6]:
test_string = 'thomas@gmil.com and mary@gmal.com'
pattern = re.compile(r'@gm[ai]l')
pattern.sub('@gmail',test_string)

'thomas@gmail.com and mary@gmail.com'

### 5. (35 pts) Pulling Book Descriptions

I've provided a url to the Google Book API (https://developers.google.com/books/) along with the isbns of the 7 Harry Potter books. 

From the API, we're going to pull the description of each book and check if their sentiment is positive or negative. The descriptions are of varying length and quality.


### Part 1

Build on the following code to create a list of dictionaries which holds a key value pair for each book and the full description. It will look something like this:

[{'book': 'book1', 'description': "Harry Potter is an ordinary boy..."}]

(Full output for this part is at the bottom of the notebook)

In [12]:
# list of isbns
isbns = ['0747532699','9780439064866','0439136369','1408855682','1338299182','0545582997','1551929783']
# you will need to concatenate each isbn with this url and use requests.get()
url = 'https://www.googleapis.com/books/v1/volumes?q=isbn:'
# create a variable list to contain the description dic
description_list = []

book = 1
for isbn in isbns:
    full_url = url + str(isbn)
    r = requests.get(full_url)
    book_json = r.json() 
    for item in book_json['items']:
        description = item['volumeInfo']['description']
    description_list.append({'book': 'book ' + str(book), 'description': description})
    book += 1      
pprint(description_list)


[{'book': 'book 1',
  'description': 'Harry Potter is an ordinary boy who lives in a cupboard '
                 "under the stairs at his Aunt Petunia and Uncle Vernon's "
                 "house, which he thinks is normal for someone like him who's "
                 "parents have been killed in a 'car crash'. He is bullied by "
                 'them and his fat, spoilt cousin Dudley, and lives a very '
                 'unremarkable life with only the odd hiccup (like his hair '
                 'growing back overnight!) to cause him much to think about. '
                 'That is until an owl turns up with a letter addressed to '
                 'Harry and all hell breaks loose! He is literally rescued by '
                 'a world where nothing is as it seems and magic lessons are '
                 'the order of the day. Read and find out how Harry discovers '
                 'his true heritage at Hogwarts School of Wizardry and '
                 'Witchcraft, the reason behi

### Part 2 

First, you'll need to install nltk at the command line and then load the packages and functions below.

Next, let's tokenize the words in each description. That will break up each description into a list of words. We'll also want to remove any words that are in stop_word set that is defined below. The new list of dictionaries looks like this:

[{'book': 'book1', 'description': ['Harry','Potter','ordinary','boy'...]}]

This one will use the word_tokenize() function.

(Full output for this part is at the bottom of the notebook)

In [8]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/steviodong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
# here is the stop word list
stop_words = set(stopwords.words("english"))             

In [13]:
# set up the variable tokenized_list list
tokenized_list = []

more_punctuation = ["``","''","--",".","...","'s"]

for i in description_list:
    tokenized_list.append({'book':i['book'], 'description':word_tokenize(i['description'])})

for i in tokenized_list:
    for word in i['description']:
        if word in stop_words:
            i['description'].remove(word)

for i in tokenized_list:
    for word in i['description']:
        if word in string.punctuation:
            i['description'].remove(word)

for i in tokenized_list:
    for word in i['description']:
        if word in more_punctuation:
            i['description'].remove(word)

tokenized_list  

[{'book': 'book 1',
  'description': ['Harry',
   'Potter',
   'ordinary',
   'boy',
   'lives',
   'cupboard',
   'stairs',
   'Aunt',
   'Petunia',
   'Uncle',
   'Vernon',
   'house',
   'thinks',
   'normal',
   'someone',
   'like',
   'parents',
   'been',
   'killed',
   "'car",
   'crash',
   'He',
   'bullied',
   'them',
   'fat',
   'spoilt',
   'cousin',
   'Dudley',
   'lives',
   'a',
   'very',
   'unremarkable',
   'life',
   'only',
   'odd',
   'hiccup',
   'like',
   'hair',
   'growing',
   'back',
   'overnight',
   ')',
   'cause',
   'much',
   'think',
   'That',
   'until',
   'an',
   'owl',
   'turns',
   'with',
   'a',
   'letter',
   'addressed',
   'Harry',
   'all',
   'hell',
   'breaks',
   'loose',
   'He',
   'literally',
   'rescued',
   'a',
   'world',
   'nothing',
   'as',
   'seems',
   'magic',
   'lessons',
   'order',
   'day',
   'Read',
   'find',
   'how',
   'Harry',
   'discovers',
   'his',
   'true',
   'heritage',
   'Hogwarts',
   '

### Part 3 

Now, let's actually check the sentiment for each book description.

Loop through the tokenized descriptions for each book and check the words against the positive_words and negative_words lists provided below. Compare the numbers of positive and negative words in each description and, for each book, print out whether the sentiment is positive or negative. If there are the same number of positive and negative words, we can call it neutral.

The output should come out along these lines:
>book1's sentiment is more negative

>book2's sentiment is more negative

>book3's sentiment is more negative

>book4's sentiment is more positive

>book5's sentiment is more positive

>book6's sentiment is more negative

>book7's sentiment is neutral

In [14]:
# read in list of positive words
positive_words = []
with open('positive-words.txt','r') as pos_file:
    for word in pos_file:
        word = word.strip()
        positive_words.append(word)

# read in list of negative words
negative_words = []
with open('negative-words.txt','r') as neg_file:
    for word in neg_file:
        word = word.strip()
        negative_words.append(word)
        
print(len(negative_words))
print(len(positive_words))

4782
2006


In [16]:
s_positive = '\'s sentiment is more positive.\n\n'
s_negative = '\'s sentiment is more negative.\n\n'
s_neutral = '\'s sentiment is more neutral.\n\n'

for item in tokenized_list:
   
    pos_results = 0
    neg_results = 0
  
    for word in item['description']:
        if word in positive_words:
            pos_results += 1 
        elif word in negative_words:
            neg_results += 1    
    if pos_results > neg_results:
        print(item['book'] + s_positive)
    elif pos_results < neg_results:
        print(item['book'] + s_negative)
    elif pos_results == neg_results:
        print(item['book'] + s_neutral)

book 1's sentiment is more negative.


book 2's sentiment is more negative.


book 3's sentiment is more negative.


book 4's sentiment is more positive.


book 5's sentiment is more positive.


book 6's sentiment is more negative.


book 7's sentiment is more neutral.




# Answers if needed 

Here are the answers to part 1 and 2 of Question 5. If you get stuck on one part, you can use these to move on to the next part of the question.

### Here is the answer to Part 1 and the input for Part 2

In [ ]:
description_list = [{'book': 'book1',
  'description': "Harry Potter is an ordinary boy who lives in a cupboard under the stairs at his Aunt Petunia and Uncle Vernon's house, which he thinks is normal for someone like him who's parents have been killed in a 'car crash'. He is bullied by them and his fat, spoilt cousin Dudley, and lives a very unremarkable life with only the odd hiccup (like his hair growing back overnight!) to cause him much to think about. That is until an owl turns up with a letter addressed to Harry and all hell breaks loose! He is literally rescued by a world where nothing is as it seems and magic lessons are the order of the day. Read and find out how Harry discovers his true heritage at Hogwarts School of Wizardry and Witchcraft, the reason behind his parents mysterious death, who is out to kill him, and how he uncovers the most amazing secret of all time, the fabled Philosopher's Stone! All this and muggles too. Now, what are they?"},
 {'book': 'book2',
  'description': 'Between the new spirit spooking his school and the mysterious forces that turn students into stone, Harry has a lot on his mind as he begins his second year at Hogwarts School of Witchcraft and Wizardry.'},
 {'book': 'book3',
  'description': "During his third year at Hogwarts School of Witchcraft and Wizardry, Harry Potter must confront the devious and dangerous wizard responsible for his parents' deaths."},
 {'book': 'book4',
  'description': "Celebrate 20 years of Harry Potter magic! The Triwizard Tournament is to be held at Hogwarts. Only wizards who are over seventeen are allowed to enter - but that doesn't stop Harry dreaming that he will win the competition. Then at Hallowe'en, when the Goblet of Fire makes its selection, Harry is amazed to find his name is one of those that the magical cup picks out. He will face death-defying tasks, dragons and Dark wizards, but with the help of his best friends, Ron and Hermione, he might just make it through - alive!These new editions of the classic and internationally bestselling, multi-award-winning series feature instantly pick-up-able new jackets by Jonny Duddle, with huge child appeal, to bring Harry Potter to the next generation of readers. It's time to PASS THE MAGIC ON ..."},
 {'book': 'book5',
  'description': "A special new edition in celebration of the 20th anniversary of the publication of Harry Potter and the Sorcerer's Stone, with a stunning new cover illustration by Caldecott Medalist Brian Selznick. There is a door at the end of a silent corridor. And it's haunting Harry Potter's dreams. Why else would he be waking in the middle of the night, screaming in terror? It's not just the upcoming O.W.L. exams; a new teacher with a personality like poisoned honey; a venomous, disgruntled house-elf; or even the growing threat of He-Who-Must-Not-Be-Named. Now Harry Potter is faced with the unreliability of the very government of the magical world and the impotence of the authorities at Hogwarts. Despite this (or perhaps because of it), he finds depth and strength in his friends, beyond what even he knew; boundless loyalty; and unbearable sacrifice. This gorgeous new edition in celebration of the 20th anniversary of the publication of Harry Potter and the Sorcerer's Stone features a newly designed cover illustrated by Caldecott Medalist Brian Selznick, as well as the beloved original interior decorations by Mary GrandPré."},
 {'book': 'book6',
  'description': 'Sixth-year Hogwarts student Harry Potter gains valuable insights into the boy Voldemort once was, even as his own world is transformed by maturing friendships, schoolwork assistance from an unexpected source, and devastating losses.'},
 {'book': 'book7',
  'description': "Harry is waiting in Privet Drive. The Order of the Phoenix is coming to escort him safely away without Voldemort and his supporters knowing--if they can. But what will Harry do then? How can he fulfil the momentous and seemingly impossible task that Professor Dumbledore has left him with?Harry has been burdened with a dark, dangerous and seemingly impossible task: that of locating and destroying Voldemort's remaining Horcruxes. Never has Harry felt so alone, or faced a future so full of shadows. But Harry must somehow find within himself the strength to complete the task he has been given. He must leave the warmth, safety and companionship of The Burrow and follow without fear or hesitation the inexorable path laid out for him ...In this final, seventh instalment of the Harry Potter series, J. K. Rowling unveils in spectacular fashion the answers to the many questions that have been so eagerly awaited. The spellbinding, richly woven narrative, which plunges, twists and turns at a breathtaking pace, confirms the author as a mistress of storytelling, whose books will be read, reread and read again."}]

### Here is the Answer to Part 2 and the input to Part 3

tokenized_list = [{'book': 'book1',
  'description': ['Harry',
   'Potter',
   'ordinary',
   'boy',
   'lives',
   'cupboard',
   'stairs',
   'Aunt',
   'Petunia',
   'Uncle',
   'Vernon',
   "'s",
   'house',
   'thinks',
   'normal',
   'someone',
   'like',
   "'s",
   'parents',
   'killed',
   "'car",
   'crash',
   'He',
   'bullied',
   'fat',
   'spoilt',
   'cousin',
   'Dudley',
   'lives',
   'unremarkable',
   'life',
   'odd',
   'hiccup',
   'like',
   'hair',
   'growing',
   'back',
   'overnight',
   'cause',
   'much',
   'think',
   'That',
   'owl',
   'turns',
   'letter',
   'addressed',
   'Harry',
   'hell',
   'breaks',
   'loose',
   'He',
   'literally',
   'rescued',
   'world',
   'nothing',
   'seems',
   'magic',
   'lessons',
   'order',
   'day',
   'Read',
   'find',
   'Harry',
   'discovers',
   'true',
   'heritage',
   'Hogwarts',
   'School',
   'Wizardry',
   'Witchcraft',
   'reason',
   'behind',
   'parents',
   'mysterious',
   'death',
   'kill',
   'uncovers',
   'amazing',
   'secret',
   'time',
   'fabled',
   'Philosopher',
   "'s",
   'Stone',
   'All',
   'muggles',
   'Now']},
 {'book': 'book2',
  'description': ['Between',
   'new',
   'spirit',
   'spooking',
   'school',
   'mysterious',
   'forces',
   'turn',
   'students',
   'stone',
   'Harry',
   'lot',
   'mind',
   'begins',
   'second',
   'year',
   'Hogwarts',
   'School',
   'Witchcraft',
   'Wizardry']},
 {'book': 'book3',
  'description': ['During',
   'third',
   'year',
   'Hogwarts',
   'School',
   'Witchcraft',
   'Wizardry',
   'Harry',
   'Potter',
   'must',
   'confront',
   'devious',
   'dangerous',
   'wizard',
   'responsible',
   'parents',
   'deaths']},
 {'book': 'book4',
  'description': ['Celebrate',
   '20',
   'years',
   'Harry',
   'Potter',
   'magic',
   'The',
   'Triwizard',
   'Tournament',
   'held',
   'Hogwarts',
   'Only',
   'wizards',
   'seventeen',
   'allowed',
   'enter',
   "n't",
   'stop',
   'Harry',
   'dreaming',
   'win',
   'competition',
   'Then',
   "Hallowe'en",
   'Goblet',
   'Fire',
   'makes',
   'selection',
   'Harry',
   'amazed',
   'find',
   'name',
   'one',
   'magical',
   'cup',
   'picks',
   'He',
   'face',
   'death-defying',
   'tasks',
   'dragons',
   'Dark',
   'wizards',
   'help',
   'best',
   'friends',
   'Ron',
   'Hermione',
   'might',
   'make',
   'alive',
   'These',
   'new',
   'editions',
   'classic',
   'internationally',
   'bestselling',
   'multi-award-winning',
   'series',
   'feature',
   'instantly',
   'pick-up-able',
   'new',
   'jackets',
   'Jonny',
   'Duddle',
   'huge',
   'child',
   'appeal',
   'bring',
   'Harry',
   'Potter',
   'next',
   'generation',
   'readers',
   'It',
   "'s",
   'time',
   'PASS',
   'THE',
   'MAGIC',
   'ON',
   '...']},
 {'book': 'book5',
  'description': ['A',
   'special',
   'new',
   'edition',
   'celebration',
   '20th',
   'anniversary',
   'publication',
   'Harry',
   'Potter',
   'Sorcerer',
   "'s",
   'Stone',
   'stunning',
   'new',
   'cover',
   'illustration',
   'Caldecott',
   'Medalist',
   'Brian',
   'Selznick',
   'There',
   'door',
   'end',
   'silent',
   'corridor',
   'And',
   "'s",
   'haunting',
   'Harry',
   'Potter',
   "'s",
   'dreams',
   'Why',
   'else',
   'would',
   'waking',
   'middle',
   'night',
   'screaming',
   'terror',
   'It',
   "'s",
   'upcoming',
   'O.W.L',
   'exams',
   'new',
   'teacher',
   'personality',
   'like',
   'poisoned',
   'honey',
   'venomous',
   'disgruntled',
   'house-elf',
   'even',
   'growing',
   'threat',
   'He-Who-Must-Not-Be-Named',
   'Now',
   'Harry',
   'Potter',
   'faced',
   'unreliability',
   'government',
   'magical',
   'world',
   'impotence',
   'authorities',
   'Hogwarts',
   'Despite',
   'perhaps',
   'finds',
   'depth',
   'strength',
   'friends',
   'beyond',
   'even',
   'knew',
   'boundless',
   'loyalty',
   'unbearable',
   'sacrifice',
   'This',
   'gorgeous',
   'new',
   'edition',
   'celebration',
   '20th',
   'anniversary',
   'publication',
   'Harry',
   'Potter',
   'Sorcerer',
   "'s",
   'Stone',
   'features',
   'newly',
   'designed',
   'cover',
   'illustrated',
   'Caldecott',
   'Medalist',
   'Brian',
   'Selznick',
   'well',
   'beloved',
   'original',
   'interior',
   'decorations',
   'Mary',
   'GrandPré']},
 {'book': 'book6',
  'description': ['Sixth-year',
   'Hogwarts',
   'student',
   'Harry',
   'Potter',
   'gains',
   'valuable',
   'insights',
   'boy',
   'Voldemort',
   'even',
   'world',
   'transformed',
   'maturing',
   'friendships',
   'schoolwork',
   'assistance',
   'unexpected',
   'source',
   'devastating',
   'losses']},
 {'book': 'book7',
  'description': ['Harry',
   'waiting',
   'Privet',
   'Drive',
   'The',
   'Order',
   'Phoenix',
   'coming',
   'escort',
   'safely',
   'away',
   'without',
   'Voldemort',
   'supporters',
   'knowing',
   '--',
   'But',
   'Harry',
   'How',
   'fulfil',
   'momentous',
   'seemingly',
   'impossible',
   'task',
   'Professor',
   'Dumbledore',
   'left',
   'Harry',
   'burdened',
   'dark',
   'dangerous',
   'seemingly',
   'impossible',
   'task',
   'locating',
   'destroying',
   'Voldemort',
   "'s",
   'remaining',
   'Horcruxes',
   'Never',
   'Harry',
   'felt',
   'alone',
   'faced',
   'future',
   'full',
   'shadows',
   'But',
   'Harry',
   'must',
   'somehow',
   'find',
   'within',
   'strength',
   'complete',
   'task',
   'given',
   'He',
   'must',
   'leave',
   'warmth',
   'safety',
   'companionship',
   'The',
   'Burrow',
   'follow',
   'without',
   'fear',
   'hesitation',
   'inexorable',
   'path',
   'laid',
   '...',
   'In',
   'final',
   'seventh',
   'instalment',
   'Harry',
   'Potter',
   'series',
   'J.',
   'K.',
   'Rowling',
   'unveils',
   'spectacular',
   'fashion',
   'answers',
   'many',
   'questions',
   'eagerly',
   'awaited',
   'The',
   'spellbinding',
   'richly',
   'woven',
   'narrative',
   'plunges',
   'twists',
   'turns',
   'breathtaking',
   'pace',
   'confirms',
   'author',
   'mistress',
   'storytelling',
   'whose',
   'books',
   'read',
   'reread',
   'read']}]